In [1]:
import pandas as pd
import csv
import geopy.distance
from operator import itemgetter


In [2]:
df = pd.read_csv('DataFrameWithClusterLabels.csv')

#### Importing avon df

In [5]:
avon_df = pd.read_csv('avon_data.csv')

In [6]:
avon_df.columns = ['index','Crime_ID', 'Month', 'Reported_by', 'Falls_within', 'Longitude', 'Latitude', 
                   'Location', 'LSOA_code', 'LSOA_name', 'Crime_type', 'Last_outcome_category', 'Context']

In [7]:
avon_df = avon_df.dropna(subset=['Longitude', 'Latitude'])

In [8]:
import decimal
def drange(x, y, jump):
    while x < y:
        yield float(x)
        x += decimal.Decimal(jump)

In [11]:
# range of coords from avon
# sorted(avon_df['Longitude'].unique())[0]
# sorted(avon_df['Longitude'].unique())[-1]

# sorted(df['Longitude'].unique())[0]
# sorted(df['Latitude'].unique())[-1] # range of these values dont change

# Construct
long = (avon_df['Longitude'].unique())
lat = (avon_df['Latitude'].unique())

In [12]:
# link to clusters
clusters_long = df['Longitude'].unique()
clusters_lat = df['Latitude'].unique()

In [13]:
intersect_long = [value for value in long if value in clusters_long]
intersect_lat = [value for value in lat if value in clusters_lat]

In [14]:
both = list(zip(intersect_long, intersect_lat))

In [15]:
long_avon = avon_df[avon_df['Longitude'].isin(intersect_long)]
lat_avon = avon_df[avon_df['Latitude'].isin(intersect_lat)]

In [16]:
## Overlap the dataframes and remove any repitive rows
## fitted for avon
final_df = pd.concat([long_avon, lat_avon]).drop_duplicates()

In [17]:
## filter with long and lat
avon_long = final_df[final_df['Longitude'].isin(clusters_long)]
avon_lat = final_df[final_df['Latitude'].isin(clusters_lat)]

In [18]:
clusters = pd.concat([avon_long, avon_lat]).drop_duplicates()
#pd.concat([clusters_long, clusters_lat]).drop_duplicates()

In [19]:
long_lat_lst = []
for i in clusters['Longitude']:
    if i in intersect_long:
        long_lat_lst.append('long')
    else:
        long_lat_lst.append('lat')

In [20]:
# avon_long['coord_value'] = 'long'
# avon_lat['coord_value'] = 'lat'

clusters['lang_lat'] = long_lat_lst

In [21]:
## Sort list based on another

result_list_long = [i for _,i in (zip(clusters['Longitude'],df['labels']))][:1476564]
result_list_lat = [i for _,i in (zip(clusters['Latitude'],df['labels']))][:116451]

In [22]:
## making seperate long and lats map to labels
long_df = df[df['Longitude'].isin(intersect_long)]
lat_df = df[df['Latitude'].isin(intersect_lat)]

In [23]:
# long_df = (df[df['Longitude'].isin(result_list_long)])

In [24]:
complete = pd.concat([long_df, lat_df])

In [26]:
clusters['class'] = result_list_long + result_list_lat

In [27]:
## opening the pollution dataframe
pollution = pd.read_csv('pollution_merged.csv')

In [28]:
## removal of uncessary rows
new_pollution = pollution[~pollution['LSOA_code'].isin(clusters['LSOA_code'].values)]

In [29]:
merged = clusters[clusters['LSOA_code'].isin(pollution['LSOA_code'].values)]

In [39]:
mer = merged[['LSOA_code']]
poll = pollution[['LSOA_code', 'pollution', 'population']]

mer_class = (mer[mer['LSOA_code'].isin(poll['LSOA_code'].values)])
# poll_class = (poll[poll['LSOA_code'].isin(merged['LSOA_code'].values)])


In [49]:
poll = (pd.concat([mer_class, poll_class])).dropna().drop_duplicates()

In [50]:
# len(merged['LSOA_code'])

final = pd.merge(poll, merged, on='LSOA_code')

In [54]:
(final).to_csv('merged_pol.csv', encoding='utf-8', index=False)

In [61]:
final['year'] = [i.year for i in pd.to_datetime(final['Month'])]

In [55]:
education = pd.read_csv('south_west_education.csv')

In [70]:
without = []
gcse = []
gcs_a = []
degree = []


for i in final['year']:
    if i == 2014:
        without.append(education['Jan 2014-Dec 2014'][0])
        gcse.append(education['Jan 2014-Dec 2014'][1])
        gcs_a.append(education['Jan 2014-Dec 2014'][2])
        degree.append(education['Jan 2014-Dec 2014'][3])
    elif i == 2015:
        without.append(education['Jan 2015-Dec 2015'][0])
        gcse.append(education['Jan 2015-Dec 2015'][1])
        gcs_a.append(education['Jan 2015-Dec 2015'][2])
        degree.append(education['Jan 2015-Dec 2015'][3])
    elif i == 2016:
        without.append(education['Jan 2016-Dec 2016'][0])
        gcse.append(education['Jan 2016-Dec 2016'][1])
        gcs_a.append(education['Jan 2016-Dec 2016'][2])
        degree.append(education['Jan 2016-Dec 2016'][3])
    elif i == 2017:
        without.append(education['Jan 2017-Dec 2017'][0])
        gcse.append(education['Jan 2017-Dec 2017'][1])
        gcs_a.append(education['Jan 2017-Dec 2017'][2])
        degree.append(education['Jan 2017-Dec 2017'][3])
        
    elif i == 2018:
        without.append(education['Jan 2018-Dec 2018'][0])
        gcse.append(education['Jan 2018-Dec 2018'][1])
        gcs_a.append(education['Jan 2018-Dec 2018'][2])
        degree.append(education['Jan 2018-Dec 2018'][3])
        
    else:
        without.append('none')
        gcse.append('none')
        gcs_a.append('none')
        degree.append('none')

In [74]:
final['without'] = without
final['gcse'] = gcse
final['gcs_a'] = gcs_a
final['degree'] = degree

In [76]:
(final).to_csv('merged_pol_2.csv', encoding='utf-8', index=False)

In [32]:
## Run based on pollution dataframe, by taking subset and 
# cluster_pollution = pd.merge(clusters, pollution, on='LSOA_code')

In [ ]:
dummy = pd.get_dummies(avon_df[['Crime_type']])

In [ ]:
## Learnt from the models as well as analysed; how can this allow us to make better tools in the future?
## these are the key finidngs and the stuff we can do with we know

## Use regression to predict the type of crime and the location? 
## The outcomes from this means that they can analyse these features in time to see how with time when this changes also
## the crime will relate and the types of crimes will also be similar and will be commited. 

In [4]:
### Spatial Regression - Convert longitdude and latidue into a degree (value), 
## Converting into the degrees in itself has limitations
## talk about feature engineering the distance
# macro-economic factors, technology, globalization, work and lifestyle choices, the organization of crime, and criminal justice resources and responses

In [27]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.geodesic(coords_1, coords_2).km)

279.35290160430094


In [28]:
pd.to_datetime(cluster_1['Month'])

0         2019-06-01
1         2019-01-01
2         2011-10-01
3         2019-05-01
4         2021-03-01
             ...    
1580217   2020-09-01
1580218   2011-06-01
1580221   2021-09-01
1580223   2017-09-01
1580224   2018-07-01
Name: Month, Length: 827344, dtype: datetime64[ns]

In [ ]:
## Regression Analysis
df['labels'].value_counts()

In [29]:
long_lat = (list(zip(avon_df['Longitude'], avon_df['Latitude'])))

In [44]:
unique_long_lat = (pd.Series(long_lat).unique())

In [ ]:
### Converts the Lat and Long of Avon coords, needed to filter DataFrame

In [48]:
pd.DataFrame(unique_long_lat).to_csv('unique_long_lat.csv', encoding='utf-8', index=False)

In [34]:
len(long_lat)

1843836

In [ ]:
to_csv()